In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


Chargement des données

In [16]:


train_df = pd.read_csv("../data/raw/train.csv")

Séparation X / y

In [17]:
X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"]


Séparation train / validation

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Préprocessing

In [19]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


Modèle 1 : Régression linéaire

In [20]:
linreg_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

linreg_model.fit(X_train, y_train)

y_pred_linreg = linreg_model.predict(X_valid)

rmse_linreg = np.sqrt(mean_squared_error(y_valid, y_pred_linreg))
r2_linreg = r2_score(y_valid, y_pred_linreg)

rmse_linreg, r2_linreg


(np.float64(30727.50707623883), 0.8769048844056192)

Modèle 2 : Random Forest

In [21]:
rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_valid)

rmse_rf = np.sqrt(mean_squared_error(y_valid, y_pred_rf))
r2_rf = r2_score(y_valid, y_pred_rf)

rmse_rf, r2_rf


(np.float64(28493.74875830324), 0.8941513442361082)

Comparaison des modèles

In [22]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest"],
    "RMSE": [rmse_linreg, rmse_rf],
    "R2": [r2_linreg, r2_rf]
})

results


,Model,RMSE,R2
0,Linear Regression,30727.507076,0.876905
1,Random Forest,28493.748758,0.894151


Conclusion générale – Modélisation

Dans le cadre de ce projet de data science appliqué au jeu de données House Prices (Ames Housing Dataset), plusieurs modèles de régression ont été développés afin de prédire le prix de vente des logements à partir de leurs caractéristiques.

Une régression linéaire a d’abord été utilisée comme modèle de référence. Ce modèle obtient des performances satisfaisantes avec une erreur moyenne de prédiction (RMSE) d’environ 30 700 dollars et un coefficient de détermination R² de 0,88, indiquant qu’il explique une grande partie de la variabilité du prix de vente.

Un modèle plus complexe de type Random Forest a ensuite été entraîné afin de capturer des relations non linéaires et des interactions entre les variables explicatives. Ce modèle améliore les performances avec un RMSE d’environ 28 500 dollars et un R² de 0,89, traduisant une meilleure précision et une capacité explicative supérieure.

La comparaison des modèles montre que le Random Forest surpasse la régression linéaire, notamment grâce à sa capacité à modéliser des structures complexes dans les données. Il a donc été retenu comme modèle final pour la prédiction du prix des maisons.

Ce projet a permis de mettre en œuvre l’ensemble des étapes clés d’un workflow de data science et de MLOps, depuis l’analyse exploratoire des données jusqu’à la modélisation, en passant par le prétraitement et l’évaluation des performances, tout en garantissant la reproductibilité et la robustesse des résultats.